## ACI COBRA SDK Exercises
---


### Created authenticated session

In [ ]:
# Disable certificate warnings
from urllib3 import disable_warnings; disable_warnings()

In [ ]:
# Setup authentication constants
APIC = 'https://sandboxapicdc.cisco.com'
USER = 'admin'
PW = 'ciscopsdt'

In [ ]:
# Import classes for authentication
from cobra.mit.access import MoDirectory
from cobra.mit.session import LoginSession

In [ ]:
# Create and setup login session
session = LoginSession(APIC, USER, PW)
moDir = MoDirectory(session)

In [ ]:
# Log on to APIC
moDir.login()

---
### Create ACI fv objects

#### Tenant, VRF, Bridge Domain, Subnet, Application Profile, EPG

In [ ]:
# Import object classes
from cobra.model.fv import Tenant, Ctx, BD, Subnet, Ap, AEPg

In [ ]:
# Create object name constants
TENANT_NAME = '514_tenant'
VRF_1_NAME = 'vrf_1'
BD_1_NAME = 'bd_1'
SUBNET_1 = '172.16.30.1/24'
APP_1_NAME = 'app_1'
EPG_1_NAME = 'epg_1'
CONTRACT_1_NAME = 'web_contract'
FILTER_1_NAME = 'web_filter'
ENTRY_1_NAME = 'web_entry'
SUBJECT_1_NAME = 'web_subject'

In [ ]:
# Create Tenant object
uniMo = moDir.lookupByDn('uni')
fvTenantMo = Tenant(uniMo, TENANT_NAME)

In [ ]:
# Create VRF object
fvCtxMo = Ctx(fvTenantMo, VRF_1_NAME)

In [ ]:
# Create Bridge Domain object
fvBDMo = BD(fvTenantMo, BD_1_NAME)

In [ ]:
# Create Subnet object
fvSubnetMo = Subnet(fvBDMo, SUBNET_1)

In [ ]:
# Create Application Profile object
fvApMo = Ap(fvTenantMo, APP_1_NAME)

In [ ]:
# Create EPG object
fvAEPgMo = AEPg(fvApMo, EPG_1_NAME)

---
### Create fv object relationships


In [ ]:
# Import object classes
from cobra.model.fv import RsCtx, RsBd

In [ ]:
# Associate Bridge Domain with VRF
RsCtx(fvBDMo, tnFvCtxName=fvCtxMo.name)

In [ ]:
# Associate EPG with Bridge Domain
RsBd(fvAEPgMo, tnFvBDName=fvBDMo.name)

---
### Create vz objects
#### Contract, Filter, Entry, Subject


In [ ]:
# Import object classes
from cobra.model.vz import Entry, Filter, BrCP, Subj, RsSubjFiltAtt

In [ ]:
# Create contract object
vzBrCPMo = BrCP(fvTenantMo, CONTRACT_1_NAME)

In [ ]:
# Create filter object
vzFilterMo = Filter(fvTenantMo, FILTER_1_NAME)

In [ ]:
# Create entry object
vzEntryMo = Entry(vzFilterMo, ENTRY_1_NAME)

In [ ]:
# Populate the entry object
vzEntryMo.dFromPort = 80
vzEntryMo.dToPort = 80
vzEntryMo.prot = 6
vzEntryMo.etherT = 'ip'

In [ ]:
# Create a subject object
vzSubjMo = Subj(vzBrCPMo, SUBJECT_1_NAME)

---
### Create vz object relationships


In [ ]:
# Associate the filter with the subject
RsSubjFiltAtt(vzSubjMo, tnVzFilterName=vzFilterMo.name)

---
### Build and send request to APIC


In [ ]:
# Import configuration object
from cobra.mit.request import ConfigRequest

In [ ]:
# Create request
cfgReq = ConfigRequest()
cfgReq.addMo(fvTenantMo)

In [ ]:
# Send request
moDir.login()
moDir.commit(cfgReq)

---
### Pause for review


In [ ]:
_ = input('Press Return/Enter to continue ')

---
### Remove objects


In [ ]:
# Mark object(s) for removal
fvTenantMo.delete()

In [ ]:
# Create request
cfgReq = ConfigRequest()
cfgReq.addMo(fvTenantMo)

In [ ]:
# Send request
moDir.login()
moDir.commit(cfgReq)